In [34]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.metrics import r2_score

%matplotlib inline

In [35]:
train = pd.read_csv("datasets/train.csv")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [36]:
def column_cleaner(data): 
    data.columns=map(str.lower, data.columns)
    data.columns = data.columns.str.strip().str.replace(" ","_")


In [37]:
column_cleaner(train)

In [38]:
def fill_na(data): 
    data[['garage_qual', 'garage_cond']] = data[['garage_qual', 'garage_cond']].fillna(value = 0)
    data['mas_vnr_area'] = data[["mas_vnr_area"]].fillna(value = 0)
    data['bsmt_qual'] = data[["bsmt_qual"]].fillna(value = 0)
    data['bsmt_cond'] = data[["bsmt_cond"]].fillna(value = 0)
    return data

In [39]:
fill_na(train)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area street  \
0      109  533352170           60        RL           NaN     13517   Pave   
1      544  531379050           60        RL          43.0     11492   Pave   
2      153  535304180           20        RL          68.0      7922   Pave   
3      318  916386060           60        RL          73.0      9802   Pave   
4      255  906425045           50        RL          82.0     14235   Pave   
...    ...        ...          ...       ...           ...       ...    ...   
2046  1587  921126030           20        RL          79.0     11449   Pave   
2047   785  905377130           30        RL           NaN     12342   Pave   
2048   916  909253010           50        RL          57.0      7558   Pave   
2049   639  535179160           20        RL          80.0     10400   Pave   
2050    10  527162130           60        RL          60.0      7500   Pave   

     alley lot_shape land_contour utilities lot_config land_slope  \
0      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
1      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
2      NaN       Reg          Lvl    AllPub     Inside        Gtl   
3      NaN       Reg          Lvl    AllPub     Inside        Gtl   
4      NaN       IR1          Lvl    AllPub     Inside        Gtl   
...    ...       ...          ...       ...        ...        ...   
2046   NaN       IR1          HLS    AllPub     Inside        Gtl   
2047   NaN       IR1          Lvl    AllPub     Inside        Gtl   
2048   NaN       Reg          Bnk    AllPub     Inside        Gtl   
2049   NaN       Reg          Lvl    AllPub     Corner        Gtl   
2050   NaN       Reg          Lvl    AllPub     Inside        Gtl   

     neighborhood condition_1 condition_2 bldg_type house_style  overall_qual  \
0          Sawyer        RRAe        Norm      1Fam      2Story             6   
1         SawyerW        Norm        Norm      1Fam      2Story             7   
2           NAmes        Norm        Norm      1Fam      1Story             5   
3          Timber        Norm        Norm      1Fam      2Story             5   
4         SawyerW        Norm        Norm      1Fam      1.5Fin             6   
...           ...         ...         ...       ...         ...           ...   
2046       Timber        Norm        Norm      1Fam      1Story             8   
2047      Edwards        Norm        Norm      1Fam      1Story             4   
2048      Crawfor        Norm        Norm      1Fam      1.5Fin             6   
2049        NAmes        Norm        Norm      1Fam      1Story             4   
2050      Gilbert        Norm        Norm      1Fam      2Story             7   

      overall_cond  year_built  year_remod/add roof_style roof_matl  \
0                8        1976            2005      Gable   CompShg   
1                5        1996            1997      Gable   CompShg   
2                7        1953            2007      Gable   CompShg   
3                5        2006            2007      Gable   CompShg   
4                8        1900            1993      Gable   CompShg   
...            ...         ...             ...        ...       ...   
2046             5        2007            2007      Gable   CompShg   
2047             5        1940            1950      Gable   CompShg   
2048             6        1928            1950      Gable   CompShg   
2049             5        1956            1956      Gable   CompShg   
2050             5        1999            1999      Gable   CompShg   

     exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area exter_qual  \
0         HdBoard      Plywood      BrkFace         289.0         Gd   
1         VinylSd      VinylSd      BrkFace         132.0         Gd   
2         VinylSd      VinylSd         None           0.0         TA   
3         VinylSd      VinylSd         None           0.0         TA   
4         Wd Sdng      Plywood         None 

In [40]:
train.drop(index = [1712], inplace = True)

In [41]:
train.loc[train["garage_cars"].isnull()].index

Int64Index([], dtype='int64')

In [42]:
train.loc[train["total_bsmt_sf"].isnull()].index

Int64Index([1327], dtype='int64')

In [43]:
train.drop(index = [1327], inplace = True)

In [44]:
train.head()

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area street alley  \
0  109  533352170           60        RL           NaN     13517   Pave   NaN   
1  544  531379050           60        RL          43.0     11492   Pave   NaN   
2  153  535304180           20        RL          68.0      7922   Pave   NaN   
3  318  916386060           60        RL          73.0      9802   Pave   NaN   
4  255  906425045           50        RL          82.0     14235   Pave   NaN   

  lot_shape land_contour utilities lot_config land_slope neighborhood  \
0       IR1          Lvl    AllPub    CulDSac        Gtl       Sawyer   
1       IR1          Lvl    AllPub    CulDSac        Gtl      SawyerW   
2       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
3       Reg          Lvl    AllPub     Inside        Gtl       Timber   
4       IR1          Lvl    AllPub     Inside        Gtl      SawyerW   

  condition_1 condition_2 bldg_type house_style  overall_qual  overall_cond  \
0        RRAe        Norm      1Fam      2Story             6             8   
1        Norm        Norm      1Fam      2Story             7             5   
2        Norm        Norm      1Fam      1Story             5             7   
3        Norm        Norm      1Fam      2Story             5             5   
4        Norm        Norm      1Fam      1.5Fin             6             8   

   year_built  year_remod/add roof_style roof_matl exterior_1st exterior_2nd  \
0        1976            2005      Gable   CompShg      HdBoard      Plywood   
1        1996            1997      Gable   CompShg      VinylSd      VinylSd   
2        1953            2007      Gable   CompShg      VinylSd      VinylSd   
3        2006            2007      Gable   CompShg      VinylSd      VinylSd   
4        1900            1993      Gable   CompShg      Wd Sdng      Plywood   

  mas_vnr_type  mas_vnr_area exter_qual exter_cond foundation bsmt_qual  \
0      BrkFace         289.0         Gd         TA     CBlock        TA   
1      BrkFace         132.0         Gd         TA      PConc        Gd   
2         None           0.0         TA         Gd     CBlock        TA   
3         None           0.0         TA         TA      PConc        Gd   
4         None           0.0         TA         TA      PConc        Fa   

  bsmt_cond bsmt_exposure bsmtfin_type_1  bsmtfin_sf_1 bsmtfin_type_2  \
0        TA            No            GLQ         533.0            Unf   
1        TA            No            GLQ         637.0            Unf   
2        TA            No            GLQ         731.0            Unf   
3        TA            No            Unf           0.0            Unf   
4        Gd            No            Unf           0.0            Unf   

   bsmtfin_sf_2  bsmt_unf_sf  total_bsmt_sf heating heating_qc central_air  \
0           0.0        192.0          725.0    GasA         Ex           Y   
1           0.0        276.0          913.0    GasA         Ex           Y   
2           0.0        326.0         1057.0    GasA         TA           Y   
3           0.0        384.0          384.0    GasA         Gd           Y   
4           0.0        676.0          676.0    GasA         TA           Y   

  electrical  1st_flr_sf  2nd_flr_sf  low_qual_fin_sf  gr_liv_area  \
0      SBrkr         725         754                0         1479   
1      SBrkr         913        1209                0         2122   
2      SBrkr        1057           0                0         1057   
3      SBrkr         744         700                0         1444   
4      SBrkr         831         614                0         1445   

   bsmt_full_bath  bsmt_half_bath  full_bath  half_bath  bedroom_abvgr  \
0             0.0             0.0          2          1              3   
1             1.0             0.0          2          1              4   
2             1.0             0.0          1          0              3   
3             0.0             0.0          2  

In [45]:
def dataframe_dictionary_input(data): 
    street_cleaned = { "Grvl" : 0,"Pave" : 1}
    data["street"] = data[["street"]].replace(street_cleaned)


    na_to_ex_rating = {"Po" : 1, "Fa" : 2,"TA" : 3, "Gd" : 4, "Ex" : 5,}
    data['bsmt_qual'] = data[['bsmt_qual']].replace(na_to_ex_rating)
    data["bsmt_cond"] = data[['bsmt_cond']].replace(na_to_ex_rating)


    heating_qc_cleaned = {"Po" : 0, "Fa" : 1,"TA" : 2, "Gd" : 3, "Ex" : 4,}
    data['heating_qc'] = data[['heating_qc']].replace(heating_qc_cleaned)

    poor_to_ex_no_na = {"Po" : 0, "Fa" : 1,"TA" : 2, "Gd" : 3, "Ex" : 4,}
    data['kitchen_qual'] = data[['kitchen_qual']].replace(poor_to_ex_no_na)

    data['exter_qual'] = data[['exter_qual']].replace(poor_to_ex_no_na)
    data['exter_cond'] = data[['exter_cond']].replace(poor_to_ex_no_na)
    
    data['garage_qual'] = data[['garage_qual']].replace(na_to_ex_rating)
    data['garage_cond'] = data[['garage_cond']].replace(na_to_ex_rating)
    data['central_air'] = data['central_air'].map({'Y': 1, "N": 0})
    
    return data 


In [46]:
dataframe_dictionary_input(train)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0      109  533352170           60        RL           NaN     13517       1   
1      544  531379050           60        RL          43.0     11492       1   
2      153  535304180           20        RL          68.0      7922       1   
3      318  916386060           60        RL          73.0      9802       1   
4      255  906425045           50        RL          82.0     14235       1   
...    ...        ...          ...       ...           ...       ...     ...   
2046  1587  921126030           20        RL          79.0     11449       1   
2047   785  905377130           30        RL           NaN     12342       1   
2048   916  909253010           50        RL          57.0      7558       1   
2049   639  535179160           20        RL          80.0     10400       1   
2050    10  527162130           60        RL          60.0      7500       1   

     alley lot_shape land_contour utilities lot_config land_slope  \
0      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
1      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
2      NaN       Reg          Lvl    AllPub     Inside        Gtl   
3      NaN       Reg          Lvl    AllPub     Inside        Gtl   
4      NaN       IR1          Lvl    AllPub     Inside        Gtl   
...    ...       ...          ...       ...        ...        ...   
2046   NaN       IR1          HLS    AllPub     Inside        Gtl   
2047   NaN       IR1          Lvl    AllPub     Inside        Gtl   
2048   NaN       Reg          Bnk    AllPub     Inside        Gtl   
2049   NaN       Reg          Lvl    AllPub     Corner        Gtl   
2050   NaN       Reg          Lvl    AllPub     Inside        Gtl   

     neighborhood condition_1 condition_2 bldg_type house_style  overall_qual  \
0          Sawyer        RRAe        Norm      1Fam      2Story             6   
1         SawyerW        Norm        Norm      1Fam      2Story             7   
2           NAmes        Norm        Norm      1Fam      1Story             5   
3          Timber        Norm        Norm      1Fam      2Story             5   
4         SawyerW        Norm        Norm      1Fam      1.5Fin             6   
...           ...         ...         ...       ...         ...           ...   
2046       Timber        Norm        Norm      1Fam      1Story             8   
2047      Edwards        Norm        Norm      1Fam      1Story             4   
2048      Crawfor        Norm        Norm      1Fam      1.5Fin             6   
2049        NAmes        Norm        Norm      1Fam      1Story             4   
2050      Gilbert        Norm        Norm      1Fam      2Story             7   

      overall_cond  year_built  year_remod/add roof_style roof_matl  \
0                8        1976            2005      Gable   CompShg   
1                5        1996            1997      Gable   CompShg   
2                7        1953            2007      Gable   CompShg   
3                5        2006            2007      Gable   CompShg   
4                8        1900            1993      Gable   CompShg   
...            ...         ...             ...        ...       ...   
2046             5        2007            2007      Gable   CompShg   
2047             5        1940            1950      Gable   CompShg   
2048             6        1928            1950      Gable   CompShg   
2049             5        1956            1956      Gable   CompShg   
2050             5        1999            1999      Gable   CompShg   

     exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area  exter_qual  \
0         HdBoard      Plywood      BrkFace         289.0           3   
1         VinylSd      VinylSd      BrkFace         132.0           3   
2         VinylSd      VinylSd         None           0.0           2   
3         VinylSd      VinylSd         None           0.0           2   
4         Wd Sdng      Plyw

In [47]:
def feature_interactions(data): 
    data["garage_comb"] = data["garage_area"] * data["garage_cars"]
    data["deck_*_porch"] = data["wood_deck_sf"] * data["open_porch_sf"]
    data["exter_cond*exter_qual"] = data["exter_cond"] * data["exter_qual"]
    data["bsmt_qual*bsmt_cond"] = data["bsmt_qual"] * data["bsmt_cond"]
    data['garage_qual*garage_con'] = data['garage_qual'] * data['garage_cond']
    return data 
    

In [48]:
feature_interactions(train)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0      109  533352170           60        RL           NaN     13517       1   
1      544  531379050           60        RL          43.0     11492       1   
2      153  535304180           20        RL          68.0      7922       1   
3      318  916386060           60        RL          73.0      9802       1   
4      255  906425045           50        RL          82.0     14235       1   
...    ...        ...          ...       ...           ...       ...     ...   
2046  1587  921126030           20        RL          79.0     11449       1   
2047   785  905377130           30        RL           NaN     12342       1   
2048   916  909253010           50        RL          57.0      7558       1   
2049   639  535179160           20        RL          80.0     10400       1   
2050    10  527162130           60        RL          60.0      7500       1   

     alley lot_shape land_contour utilities lot_config land_slope  \
0      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
1      NaN       IR1          Lvl    AllPub    CulDSac        Gtl   
2      NaN       Reg          Lvl    AllPub     Inside        Gtl   
3      NaN       Reg          Lvl    AllPub     Inside        Gtl   
4      NaN       IR1          Lvl    AllPub     Inside        Gtl   
...    ...       ...          ...       ...        ...        ...   
2046   NaN       IR1          HLS    AllPub     Inside        Gtl   
2047   NaN       IR1          Lvl    AllPub     Inside        Gtl   
2048   NaN       Reg          Bnk    AllPub     Inside        Gtl   
2049   NaN       Reg          Lvl    AllPub     Corner        Gtl   
2050   NaN       Reg          Lvl    AllPub     Inside        Gtl   

     neighborhood condition_1 condition_2 bldg_type house_style  overall_qual  \
0          Sawyer        RRAe        Norm      1Fam      2Story             6   
1         SawyerW        Norm        Norm      1Fam      2Story             7   
2           NAmes        Norm        Norm      1Fam      1Story             5   
3          Timber        Norm        Norm      1Fam      2Story             5   
4         SawyerW        Norm        Norm      1Fam      1.5Fin             6   
...           ...         ...         ...       ...         ...           ...   
2046       Timber        Norm        Norm      1Fam      1Story             8   
2047      Edwards        Norm        Norm      1Fam      1Story             4   
2048      Crawfor        Norm        Norm      1Fam      1.5Fin             6   
2049        NAmes        Norm        Norm      1Fam      1Story             4   
2050      Gilbert        Norm        Norm      1Fam      2Story             7   

      overall_cond  year_built  year_remod/add roof_style roof_matl  \
0                8        1976            2005      Gable   CompShg   
1                5        1996            1997      Gable   CompShg   
2                7        1953            2007      Gable   CompShg   
3                5        2006            2007      Gable   CompShg   
4                8        1900            1993      Gable   CompShg   
...            ...         ...             ...        ...       ...   
2046             5        2007            2007      Gable   CompShg   
2047             5        1940            1950      Gable   CompShg   
2048             6        1928            1950      Gable   CompShg   
2049             5        1956            1956      Gable   CompShg   
2050             5        1999            1999      Gable   CompShg   

     exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area  exter_qual  \
0         HdBoard      Plywood      BrkFace         289.0           3   
1         VinylSd      VinylSd      BrkFace         132.0           3   
2         VinylSd      VinylSd         None           0.0           2   
3         VinylSd      VinylSd         None           0.0           2   
4         Wd Sdng      Plyw

In [49]:
def dummy_cols_maker(data):
    data = pd.get_dummies(data, columns = ['neighborhood'], drop_first=True)
    data = pd.get_dummies(data, columns = ['electrical'], drop_first = True)
    data = pd.get_dummies(data, columns = ['heating'], drop_first = True)
    data = pd.get_dummies(data, columns = ['functional'], drop_first = True)
    data = pd.get_dummies(data, columns = ['ms_zoning'], drop_first = True) 
    data = pd.get_dummies(data, columns = ['utilities'], drop_first = True)
    data = pd.get_dummies(data, columns = ['condition_1'], drop_first = True)

    
    return data

In [50]:
train = dummy_cols_maker(data = train) 

In [51]:
train.head()

id        pid  ms_subclass  lot_frontage  lot_area  street alley  \
0  109  533352170           60           NaN     13517       1   NaN   
1  544  531379050           60          43.0     11492       1   NaN   
2  153  535304180           20          68.0      7922       1   NaN   
3  318  916386060           60          73.0      9802       1   NaN   
4  255  906425045           50          82.0     14235       1   NaN   

  lot_shape land_contour lot_config land_slope condition_2 bldg_type  \
0       IR1          Lvl    CulDSac        Gtl        Norm      1Fam   
1       IR1          Lvl    CulDSac        Gtl        Norm      1Fam   
2       Reg          Lvl     Inside        Gtl        Norm      1Fam   
3       Reg          Lvl     Inside        Gtl        Norm      1Fam   
4       IR1          Lvl     Inside        Gtl        Norm      1Fam   

  house_style  overall_qual  overall_cond  year_built  year_remod/add  \
0      2Story             6             8        1976            2005   
1      2Story             7             5        1996            1997   
2      1Story             5             7        1953            2007   
3      2Story             5             5        2006            2007   
4      1.5Fin             6             8        1900            1993   

  roof_style roof_matl exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area  \
0      Gable   CompShg      HdBoard      Plywood      BrkFace         289.0   
1      Gable   CompShg      VinylSd      VinylSd      BrkFace         132.0   
2      Gable   CompShg      VinylSd      VinylSd         None           0.0   
3      Gable   CompShg      VinylSd      VinylSd         None           0.0   
4      Gable   CompShg      Wd Sdng      Plywood         None           0.0   

   exter_qual  exter_cond foundation  bsmt_qual  bsmt_cond bsmt_exposure  \
0           3           2     CBlock          3          3            No   
1           3           2      PConc          4          3            No   
2           2           3     CBlock          3          3            No   
3           2           2      PConc          4          3            No   
4           2           2      PConc          2          4            No   

  bsmtfin_type_1  bsmtfin_sf_1 bsmtfin_type_2  bsmtfin_sf_2  bsmt_unf_sf  \
0            GLQ         533.0            Unf           0.0        192.0   
1            GLQ         637.0            Unf           0.0        276.0   
2            GLQ         731.0            Unf           0.0        326.0   
3            Unf           0.0            Unf           0.0        384.0   
4            Unf           0.0            Unf           0.0        676.0   

   total_bsmt_sf  heating_qc  central_air  1st_flr_sf  2nd_flr_sf  \
0          725.0           4            1         725         754   
1          913.0           4            1         913        1209   
2         1057.0           2            1        1057           0   
3          384.0           3            1         744         700   
4          676.0           2            1         831         614   

   low_qual_fin_sf  gr_liv_area  bsmt_full_bath  bsmt_half_bath  full_bath  \
0                0         1479             0.0             0.0          2   
1                0         2122             1.0             0.0          2   
2                0         1057             1.0             0.0          1   
3                0         1444             0.0             0.0          2   
4                0         1445             0.0             0.0          2   

   half_bath  bedroom_abvgr  kitchen_abvgr  kitchen_qual  totrms_abvgrd  \
0          1              3              1             3              6   
1          1              4              1             3              8   
2          0              3              1             3              5   
3          1              3              1             2              7   
4          0              3              1         

In [52]:
plt.figure(figsize = (30,30))
sns.heatmap(train.corr()[['saleprice']].sort_values(by = "saleprice" ,ascending = False), annot = True, cmap = 'coolwarm');

In [53]:
train.columns[-58:]

Index(['neighborhood_Blueste', 'neighborhood_BrDale', 'neighborhood_BrkSide',
       'neighborhood_ClearCr', 'neighborhood_CollgCr', 'neighborhood_Crawfor',
       'neighborhood_Edwards', 'neighborhood_Gilbert', 'neighborhood_Greens',
       'neighborhood_GrnHill', 'neighborhood_IDOTRR', 'neighborhood_Landmrk',
       'neighborhood_MeadowV', 'neighborhood_Mitchel', 'neighborhood_NAmes',
       'neighborhood_NPkVill', 'neighborhood_NWAmes', 'neighborhood_NoRidge',
       'neighborhood_NridgHt', 'neighborhood_OldTown', 'neighborhood_SWISU',
       'neighborhood_Sawyer', 'neighborhood_SawyerW', 'neighborhood_Somerst',
       'neighborhood_StoneBr', 'neighborhood_Timber', 'neighborhood_Veenker',
       'electrical_FuseF', 'electrical_FuseP', 'electrical_Mix',
       'electrical_SBrkr', 'heating_GasW', 'heating_Grav', 'heating_OthW',
       'heating_Wall', 'functional_Maj2', 'functional_Min1', 'functional_Min2',
       'functional_Mod', 'functional_Sal', 'functional_Sev', 'functional_Typ',


In [54]:
features = ['overall_qual', 'exter_qual', 'gr_liv_area', 'garage_comb', 'kitchen_qual', 'total_bsmt_sf',
              'deck_*_porch', 'exter_cond*exter_qual', 'wood_deck_sf', 'fireplaces', 'full_bath','bsmt_qual*bsmt_cond',
            'mas_vnr_area', 'open_porch_sf', 'year_built', 'year_remod/add', 
       'neighborhood_Blueste', 'neighborhood_BrDale', 'neighborhood_BrkSide',
       'neighborhood_ClearCr', 'neighborhood_CollgCr', 'neighborhood_Crawfor',
       'neighborhood_Edwards', 'neighborhood_Gilbert', 'neighborhood_Greens',
       'neighborhood_GrnHill', 'neighborhood_IDOTRR', 'neighborhood_Landmrk',
       'neighborhood_MeadowV', 'neighborhood_Mitchel', 'neighborhood_NAmes',
       'neighborhood_NPkVill', 'neighborhood_NWAmes', 'neighborhood_NoRidge',
       'neighborhood_NridgHt', 'neighborhood_OldTown', 'neighborhood_SWISU',
       'neighborhood_Sawyer', 'neighborhood_SawyerW', 'neighborhood_Somerst',
       'neighborhood_StoneBr', 'neighborhood_Timber', 'neighborhood_Veenker',
       'electrical_FuseF', 'electrical_FuseP', 'electrical_Mix',
       'electrical_SBrkr', 'heating_GasW', 'heating_Grav', 'heating_OthW',
       'heating_Wall', 'functional_Maj2', 'functional_Min1', 'functional_Min2',
       'functional_Mod', 'functional_Sal', 'functional_Sev', 'functional_Typ',
       'ms_zoning_C (all)', 'ms_zoning_FV', 'ms_zoning_I (all)',
       'ms_zoning_RH', 'ms_zoning_RL', 'ms_zoning_RM', 'utilities_NoSeWa',
       'utilities_NoSewr', 'condition_1_Feedr', 'condition_1_Norm',
       'condition_1_PosA', 'condition_1_PosN', 'condition_1_RRAe',
       'condition_1_RRAn', 'condition_1_RRNe', 'condition_1_RRNn', 'street']
             

In [55]:
X = train[features]
y = train['saleprice']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [57]:
pt = PowerTransformer()
pt.fit(X_train)
X_train_pt_transformed = pt.transform(X_train)
X_test_pt_transformed = pt.transform(X_test)
    
    
pt_y = PowerTransformer()
pt_y.fit(y_train.to_frame())
y_train_pt_transformed = pt_y.transform(y_train.to_frame())
y_test_pt_transformed = pt_y.transform(y_test.to_frame())
    
    

/Users/nathan/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2798: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [61]:
#def fitter_and_instantiator(): 
lr = LinearRegression()  

In [62]:
lasso = LassoCV(n_alphas=200, cv=5)

In [63]:
ridge = RidgeCV(alphas=np.linspace(.1, 10, 100))

In [64]:
lr_scores = cross_val_score(lr, X_train, y_train_pt_transformed, cv=3)
lr_scores.mean()

0.8354040356739959

In [65]:
lasso_scores = cross_val_score(lasso, X_train, y_train_pt_transformed[:, 0], cv=3)
lasso_scores.mean()

0.6718197013111511

In [66]:
ridge_scores = cross_val_score(ridge, X_train, y_train_pt_transformed, cv=3)
ridge_scores.mean()

0.8368478049484223

In [67]:
lr.fit(X_train, y_train_pt_transformed)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [68]:
lr.score(X_train, y_train_pt_transformed)

0.8887731511310196

In [69]:
lr.score(X_test, y_test_pt_transformed)

0.8643078480974979

In [70]:
ridge.fit(X_train, y_train_pt_transformed)

RidgeCV(alphas=array([ 0.1,  0.2, ...,  9.9, 10. ]), cv=None,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [71]:
ridge.score(X_train, y_train_pt_transformed)

0.8883254511763016

In [72]:
ridge.score(X_test, y_test_pt_transformed)

0.8654366001123978

In [73]:
pred = ridge.predict(X_test)

In [74]:
pred_reversed = pt_y.inverse_transform(pred.reshape(-1,1))

In [75]:
r2_score(y_test, pred_reversed)

0.8959802702905767

In [77]:
test= pd.read_csv('datasets/test.csv')

In [78]:
column_cleaner(test)

In [79]:
dataframe_dictionary_input(test)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0    2658  902301120          190        RM          69.0      9142       1   
1    2718  905108090           90        RL           NaN      9662       1   
2    2414  528218130           60        RL          58.0     17104       1   
3    1989  902207150           30        RM          60.0      8520       1   
4     625  535105100           20        RL           NaN      9500       1   
..    ...        ...          ...       ...           ...       ...     ...   
874  1662  527377110           60        RL          80.0      8000       1   
875  1234  535126140           60        RL          90.0     14670       1   
876  1373  904100040           20        RL          55.0      8250       1   
877  1672  527425140           20        RL          60.0      9000       1   
878  1939  535327160           20        RL          70.0      8400       1   

    alley lot_shape land_contour utilities lot_config land_slope neighborhood  \
0    Grvl       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
1     NaN       IR1          Lvl    AllPub     Inside        Gtl       Sawyer   
2     NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   
3     NaN       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
4     NaN       IR1          Lvl    AllPub     Inside        Gtl        NAmes   
..    ...       ...          ...       ...        ...        ...          ...   
874   NaN       Reg          Lvl    AllPub     Inside        Gtl       NWAmes   
875   NaN       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
876   NaN       Reg          Lvl    AllPub     Inside        Gtl       Sawyer   
877   NaN       Reg          Lvl    AllPub        FR2        Gtl        NAmes   
878   NaN       Reg          Lvl    AllPub     Corner        Gtl        NAmes   

    condition_1 condition_2 bldg_type house_style  overall_qual  overall_cond  \
0          Norm        Norm    2fmCon      2Story             6             8   
1          Norm        Norm    Duplex      1Story             5             4   
2          Norm        Norm      1Fam      2Story             7             5   
3          Norm        Norm      1Fam      1Story             5             6   
4          Norm        Norm      1Fam      1Story             6             5   
..          ...         ...       ...         ...           ...           ...   
874        PosN        Norm      1Fam      2Story             6             6   
875        Norm        Norm      1Fam      2Story             6             7   
876       Feedr        Norm      1Fam      1Story             5             5   
877        Norm        Norm      1Fam      1Story             4             6   
878        Norm        Norm      1Fam      1Story             5             5   

     year_built  year_remod/add roof_style roof_matl exterior_1st  \
0          1910            1950      Gable   CompShg      AsbShng   
1          1977            1977      Gable   CompShg      Plywood   
2          2006            2006      Gable   CompShg      VinylSd   
3          1923            2006      Gable   CompShg      Wd Sdng   
4          1963            1963      Gable   CompShg      Plywood   
..          ...             ...        ...       ...          ...   
874        1974            1974      Gable   CompShg      HdBoard   
875        1966            1999      Gable   CompShg      VinylSd   
876        1968            1968        Hip   CompShg      HdBoard   
877        1971            1971      Gable   CompShg      HdBoard   
878        1955            1955      Gable   CompShg      MetalSd   

    exterior_2nd mas_vnr_type  mas_vnr_area  exter_qual  exter_cond  \
0        AsbShng         None           0.0           2           1   
1        Plywood         None           0.0           2           2   
2        VinylSd         None           0.0           3           2   

In [80]:
fill_na(test)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0    2658  902301120          190        RM          69.0      9142       1   
1    2718  905108090           90        RL           NaN      9662       1   
2    2414  528218130           60        RL          58.0     17104       1   
3    1989  902207150           30        RM          60.0      8520       1   
4     625  535105100           20        RL           NaN      9500       1   
..    ...        ...          ...       ...           ...       ...     ...   
874  1662  527377110           60        RL          80.0      8000       1   
875  1234  535126140           60        RL          90.0     14670       1   
876  1373  904100040           20        RL          55.0      8250       1   
877  1672  527425140           20        RL          60.0      9000       1   
878  1939  535327160           20        RL          70.0      8400       1   

    alley lot_shape land_contour utilities lot_config land_slope neighborhood  \
0    Grvl       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
1     NaN       IR1          Lvl    AllPub     Inside        Gtl       Sawyer   
2     NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   
3     NaN       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
4     NaN       IR1          Lvl    AllPub     Inside        Gtl        NAmes   
..    ...       ...          ...       ...        ...        ...          ...   
874   NaN       Reg          Lvl    AllPub     Inside        Gtl       NWAmes   
875   NaN       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
876   NaN       Reg          Lvl    AllPub     Inside        Gtl       Sawyer   
877   NaN       Reg          Lvl    AllPub        FR2        Gtl        NAmes   
878   NaN       Reg          Lvl    AllPub     Corner        Gtl        NAmes   

    condition_1 condition_2 bldg_type house_style  overall_qual  overall_cond  \
0          Norm        Norm    2fmCon      2Story             6             8   
1          Norm        Norm    Duplex      1Story             5             4   
2          Norm        Norm      1Fam      2Story             7             5   
3          Norm        Norm      1Fam      1Story             5             6   
4          Norm        Norm      1Fam      1Story             6             5   
..          ...         ...       ...         ...           ...           ...   
874        PosN        Norm      1Fam      2Story             6             6   
875        Norm        Norm      1Fam      2Story             6             7   
876       Feedr        Norm      1Fam      1Story             5             5   
877        Norm        Norm      1Fam      1Story             4             6   
878        Norm        Norm      1Fam      1Story             5             5   

     year_built  year_remod/add roof_style roof_matl exterior_1st  \
0          1910            1950      Gable   CompShg      AsbShng   
1          1977            1977      Gable   CompShg      Plywood   
2          2006            2006      Gable   CompShg      VinylSd   
3          1923            2006      Gable   CompShg      Wd Sdng   
4          1963            1963      Gable   CompShg      Plywood   
..          ...             ...        ...       ...          ...   
874        1974            1974      Gable   CompShg      HdBoard   
875        1966            1999      Gable   CompShg      VinylSd   
876        1968            1968        Hip   CompShg      HdBoard   
877        1971            1971      Gable   CompShg      HdBoard   
878        1955            1955      Gable   CompShg      MetalSd   

    exterior_2nd mas_vnr_type  mas_vnr_area  exter_qual  exter_cond  \
0        AsbShng         None           0.0           2           1   
1        Plywood         None           0.0           2           2   
2        VinylSd         None           0.0           3           2   

In [81]:
feature_interactions(test)

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0    2658  902301120          190        RM          69.0      9142       1   
1    2718  905108090           90        RL           NaN      9662       1   
2    2414  528218130           60        RL          58.0     17104       1   
3    1989  902207150           30        RM          60.0      8520       1   
4     625  535105100           20        RL           NaN      9500       1   
..    ...        ...          ...       ...           ...       ...     ...   
874  1662  527377110           60        RL          80.0      8000       1   
875  1234  535126140           60        RL          90.0     14670       1   
876  1373  904100040           20        RL          55.0      8250       1   
877  1672  527425140           20        RL          60.0      9000       1   
878  1939  535327160           20        RL          70.0      8400       1   

    alley lot_shape land_contour utilities lot_config land_slope neighborhood  \
0    Grvl       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
1     NaN       IR1          Lvl    AllPub     Inside        Gtl       Sawyer   
2     NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   
3     NaN       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
4     NaN       IR1          Lvl    AllPub     Inside        Gtl        NAmes   
..    ...       ...          ...       ...        ...        ...          ...   
874   NaN       Reg          Lvl    AllPub     Inside        Gtl       NWAmes   
875   NaN       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
876   NaN       Reg          Lvl    AllPub     Inside        Gtl       Sawyer   
877   NaN       Reg          Lvl    AllPub        FR2        Gtl        NAmes   
878   NaN       Reg          Lvl    AllPub     Corner        Gtl        NAmes   

    condition_1 condition_2 bldg_type house_style  overall_qual  overall_cond  \
0          Norm        Norm    2fmCon      2Story             6             8   
1          Norm        Norm    Duplex      1Story             5             4   
2          Norm        Norm      1Fam      2Story             7             5   
3          Norm        Norm      1Fam      1Story             5             6   
4          Norm        Norm      1Fam      1Story             6             5   
..          ...         ...       ...         ...           ...           ...   
874        PosN        Norm      1Fam      2Story             6             6   
875        Norm        Norm      1Fam      2Story             6             7   
876       Feedr        Norm      1Fam      1Story             5             5   
877        Norm        Norm      1Fam      1Story             4             6   
878        Norm        Norm      1Fam      1Story             5             5   

     year_built  year_remod/add roof_style roof_matl exterior_1st  \
0          1910            1950      Gable   CompShg      AsbShng   
1          1977            1977      Gable   CompShg      Plywood   
2          2006            2006      Gable   CompShg      VinylSd   
3          1923            2006      Gable   CompShg      Wd Sdng   
4          1963            1963      Gable   CompShg      Plywood   
..          ...             ...        ...       ...          ...   
874        1974            1974      Gable   CompShg      HdBoard   
875        1966            1999      Gable   CompShg      VinylSd   
876        1968            1968        Hip   CompShg      HdBoard   
877        1971            1971      Gable   CompShg      HdBoard   
878        1955            1955      Gable   CompShg      MetalSd   

    exterior_2nd mas_vnr_type  mas_vnr_area  exter_qual  exter_cond  \
0        AsbShng         None           0.0           2           1   
1        Plywood         None           0.0           2           2   
2        VinylSd         None           0.0           3           2   

In [82]:
test.head()

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area  street  \
0  2658  902301120          190        RM          69.0      9142       1   
1  2718  905108090           90        RL           NaN      9662       1   
2  2414  528218130           60        RL          58.0     17104       1   
3  1989  902207150           30        RM          60.0      8520       1   
4   625  535105100           20        RL           NaN      9500       1   

  alley lot_shape land_contour utilities lot_config land_slope neighborhood  \
0  Grvl       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
1   NaN       IR1          Lvl    AllPub     Inside        Gtl       Sawyer   
2   NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   
3   NaN       Reg          Lvl    AllPub     Inside        Gtl      OldTown   
4   NaN       IR1          Lvl    AllPub     Inside        Gtl        NAmes   

  condition_1 condition_2 bldg_type house_style  overall_qual  overall_cond  \
0        Norm        Norm    2fmCon      2Story             6             8   
1        Norm        Norm    Duplex      1Story             5             4   
2        Norm        Norm      1Fam      2Story             7             5   
3        Norm        Norm      1Fam      1Story             5             6   
4        Norm        Norm      1Fam      1Story             6             5   

   year_built  year_remod/add roof_style roof_matl exterior_1st exterior_2nd  \
0        1910            1950      Gable   CompShg      AsbShng      AsbShng   
1        1977            1977      Gable   CompShg      Plywood      Plywood   
2        2006            2006      Gable   CompShg      VinylSd      VinylSd   
3        1923            2006      Gable   CompShg      Wd Sdng      Wd Sdng   
4        1963            1963      Gable   CompShg      Plywood      Plywood   

  mas_vnr_type  mas_vnr_area  exter_qual  exter_cond foundation  bsmt_qual  \
0         None           0.0           2           1      Stone        2.0   
1         None           0.0           2           2     CBlock        4.0   
2         None           0.0           3           2      PConc        4.0   
3         None           0.0           3           2     CBlock        3.0   
4      BrkFace         247.0           2           2     CBlock        4.0   

   bsmt_cond bsmt_exposure bsmtfin_type_1  bsmtfin_sf_1 bsmtfin_type_2  \
0          3            No            Unf             0            Unf   
1          3            No            Unf             0            Unf   
2          4            Av            GLQ           554            Unf   
3          3            No            Unf             0            Unf   
4          3            No            BLQ           609            Unf   

   bsmtfin_sf_2  bsmt_unf_sf  total_bsmt_sf heating  heating_qc  central_air  \
0             0         1020           1020    GasA           3            0   
1             0         1967           1967    GasA           2            1   
2             0          100            654    GasA           4            1   
3             0          968            968    GasA           2            1   
4             0          785           1394    GasA           3            1   

  electrical  1st_flr_sf  2nd_flr_sf  low_qual_fin_sf  gr_liv_area  \
0      FuseP         908        1020                0         1928   
1      SBrkr        1967           0                0         1967   
2      SBrkr         664         832                0         1496   
3      SBrkr         968           0                0          968   
4      SBrkr        1394           0                0         1394   

   bsmt_full_bath  bsmt_half_bath  full_bath  half_bath  bedroom_abvgr  \
0               0               0          2          0              4   
1               0               0          2          0              6   
2               1               0          2          1              3   

In [83]:

test = dummy_cols_maker(data = test) 

In [84]:
test.head()

id        pid  ms_subclass  lot_frontage  lot_area  street alley  \
0  2658  902301120          190          69.0      9142       1  Grvl   
1  2718  905108090           90           NaN      9662       1   NaN   
2  2414  528218130           60          58.0     17104       1   NaN   
3  1989  902207150           30          60.0      8520       1   NaN   
4   625  535105100           20           NaN      9500       1   NaN   

  lot_shape land_contour lot_config land_slope condition_2 bldg_type  \
0       Reg          Lvl     Inside        Gtl        Norm    2fmCon   
1       IR1          Lvl     Inside        Gtl        Norm    Duplex   
2       IR1          Lvl     Inside        Gtl        Norm      1Fam   
3       Reg          Lvl     Inside        Gtl        Norm      1Fam   
4       IR1          Lvl     Inside        Gtl        Norm      1Fam   

  house_style  overall_qual  overall_cond  year_built  year_remod/add  \
0      2Story             6             8        1910            1950   
1      1Story             5             4        1977            1977   
2      2Story             7             5        2006            2006   
3      1Story             5             6        1923            2006   
4      1Story             6             5        1963            1963   

  roof_style roof_matl exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area  \
0      Gable   CompShg      AsbShng      AsbShng         None           0.0   
1      Gable   CompShg      Plywood      Plywood         None           0.0   
2      Gable   CompShg      VinylSd      VinylSd         None           0.0   
3      Gable   CompShg      Wd Sdng      Wd Sdng         None           0.0   
4      Gable   CompShg      Plywood      Plywood      BrkFace         247.0   

   exter_qual  exter_cond foundation  bsmt_qual  bsmt_cond bsmt_exposure  \
0           2           1      Stone        2.0          3            No   
1           2           2     CBlock        4.0          3            No   
2           3           2      PConc        4.0          4            Av   
3           3           2     CBlock        3.0          3            No   
4           2           2     CBlock        4.0          3            No   

  bsmtfin_type_1  bsmtfin_sf_1 bsmtfin_type_2  bsmtfin_sf_2  bsmt_unf_sf  \
0            Unf             0            Unf             0         1020   
1            Unf             0            Unf             0         1967   
2            GLQ           554            Unf             0          100   
3            Unf             0            Unf             0          968   
4            BLQ           609            Unf             0          785   

   total_bsmt_sf  heating_qc  central_air  1st_flr_sf  2nd_flr_sf  \
0           1020           3            0         908        1020   
1           1967           2            1        1967           0   
2            654           4            1         664         832   
3            968           2            1         968           0   
4           1394           3            1        1394           0   

   low_qual_fin_sf  gr_liv_area  bsmt_full_bath  bsmt_half_bath  full_bath  \
0                0         1928               0               0          2   
1                0         1967               0               0          2   
2                0         1496               1               0          2   
3                0          968               0               0          1   
4                0         1394               1               0          1   

   half_bath  bedroom_abvgr  kitchen_abvgr  kitchen_qual  totrms_abvgrd  \
0          0              4              2             1              9   
1          0              6              2             2             10   
2          1              3              1             3              7   
3          0              2              1             2              5   
4          1              3              1    

In [85]:
test['neighborhood_GrnHill'] = 0 
test['electrical_Mix'] = 0 
test['neighborhood_Landmrk'] = 0
test['heating_Wall'] = 0 
test['heating_OthW'] = 0 
test['functional_Sal'] = 0
test['functional_Sev'] =0 
test['utilities_NoSeWa'] =0 
test['ms_zoning_C (all)'] = 0 

In [86]:
test[features]

overall_qual  exter_qual  gr_liv_area  garage_comb  kitchen_qual  \
0               6           2         1928          440             1   
1               5           2         1967         1160             2   
2               7           3         1496          852             3   
3               5           3          968          960             2   
4               6           2         1394         1028             2   
..            ...         ...          ...          ...           ...   
874             6           2         1877          976             2   
875             6           3         1988          960             3   
876             5           2         1211          322             2   
877             4           2          864         1056             2   
878             5           2          925          390             2   

     total_bsmt_sf  deck_*_porch  exter_cond*exter_qual  wood_deck_sf  \
0             1020             0                      2             0   
1             1967             0                      4           170   
2              654          2400                      6           100   
3              968             0                      6             0   
4             1394             0                      4             0   
..             ...           ...                    ...           ...   
874           1084             0                      4             0   
875           1104             0                      9             0   
876            952             0                      4             0   
877            864             0                      4             0   
878            923             0                      6            81   

     fireplaces  full_bath  bsmt_qual*bsmt_cond  mas_vnr_area  open_porch_sf  \
0             0          2                  6.0           0.0             60   
1             0          2                 12.0           0.0              0   
2             1          2                 16.0           0.0             24   
3             0          1                  9.0           0.0              0   
4             2          1                 12.0         247.0             76   
..          ...        ...                  ...           ...            ...   
874           1          2                  9.0           0.0             96   
875           1          2                  9.0         410.0            230   
876           1          1                  9.0           0.0             63   
877           0          1                  9.0           0.0              0   
878           2          1                  9.0           0.0              0   

     year_built  year_remod/add  neighborhood_Blueste  neighborhood_BrDale  \
0          1910            1950                     0                    0   
1          1977            1977                     0                    0   
2          2006            2006                     0                    0   
3          1923            2006                     0                    0   
4          1963            1963                     0                    0   
..          ...             ...                   ...                  ...   
874        1974            1974                     0                    0   
875        1966            1999                     0                    0   
876        1968            1968                     0                    0   
877        1971            1971                     0                    0   
878        1955            1955                     0                    0   

     neighborhood_BrkSide  neighborhood_ClearCr  neighborhood_CollgCr  \
0                       0                     0                     0   
1                       0                     0                     0   
2                       0                     0                     0   
3                       0                     0              

In [87]:
#def prediction_maker(data): 
test_preds = ridge.predict(test[features])
pred_reversed = pt_y.inverse_transform(test_preds.reshape(-1,1))
test['preds_1'] = pred_reversed

submission_10 = test.loc[:, ['id', 'preds_1']]
submission_10.rename(columns = {'id': 'Id', 'preds_1': 'SalePrice'}, inplace=True)
    

In [88]:
submission_10.head()
   

Id      SalePrice
0  2658  109254.619094
1  2718  181468.702350
2  2414  202829.268664
3  1989  111695.938128
4   625  174592.126285

In [89]:
submission_10.to_csv('./Submissions/submissions_10', index=False )